I called SNPs across all species. Now, for each species, find the major allele to create a pseudo reference where the original loblolly reference allele is replaced by the species-specific major allele.

Afterward, remap and call SNPs for each species.

In [1]:
from pythonimports import *
DIR = '/home/lindb/eckertlab/BURT/002_trim_and_map/loblolly_mapping/BURT'
vcfdir = op.join(DIR, '04_vcfs')

In [5]:
lview,dview = get_client('xmn')

33 33


In [ ]:
# gunzip -c TotalRawSNPs.txt.gz > TotalRawSNPs.txt

In [2]:
def get_client(profile='default', **kwargs) -> tuple:
    """Get lview,dview from ipcluster."""
    rc = Client(profile=profile, **kwargs)
    dview = rc[:]
    lview = rc.load_balanced_view()
    print(len(lview),len(dview))
    return lview, dview

In [ ]:
# from 003_snp_calling
df = parallel_read(op.join(vcfdir, 'TotalRawSNPs.txt'), linenums=2142767, lview=lview)

parallel_read() is:
	determining chunksize (nrows) ...
	sending jobs to engines ...


sending jobs: 100%|██████████| 34/34 [00:00<00:00, 596.91it/s]



Watching 34 parallel_read() jobs ...


TotalRawSNPs.txt:   0%|          | 0/34 [00:00<?, ?it/s]